In [1]:
import os
import pandas as pd
from biopandas.pdb import PandasPdb
from Bio.SeqUtils import seq1
from pandaprot import PandaProt
import glob

In [2]:
def extract_sequence_from_pdb(pdb_df:PandasPdb, chain_id:str, record_name='ATOM'):
    """
    Extracts the sequence of a specified chain from a BioPandas PDB DataFrame.
    Args:
        pdb_df (PandasPdb): A PandasPdb object containing the PDB data.
        chain_id (str): The chain ID for which to extract the sequence.
        record_name (str): The type of record to extract the sequence from ('SEQRES' or 'ATOM').

    Returns:
        str: The single-letter amino acid sequence of the specified chain.
    """
    
    ## Get sequence from SEQRES records
    if record_name == 'SEQRES':
        other_records = pdb_df.df['OTHERS']
        seqres_records = other_records[other_records['record_name'] == 'SEQRES']

        sequence = ''
        for index, row in seqres_records.iterrows():
            entry = row['entry'].split(" ")
            print(entry)
            id = entry[3]
            chain_id = entry[4]
            residues = entry[8:]

            if chain_id == chain_id:
                for residue in residues:
                    sequence += seq1(residue)
        return sequence
    
    ## Get sequence from ATOM records
    elif record_name == 'ATOM':
        sequence = ''
        seen_residues = set()
        for index, row in pdb_df.df['ATOM'].iterrows():
            resn = row['residue_number']
            if row['chain_id'] == chain_id and resn not in seen_residues:
                seen_residues.add(resn)
                sequence += seq1(row['residue_name'])
        return sequence
    
    else:
        raise ValueError("Invalid record name. Use 'SEQRES' or 'ATOM'.")

In [3]:
## Setup DataFrame to store sequences
sequences_df = pd.DataFrame(columns=["pdb_id", "h_chain_id", "l_chain_id", "antigen_ids", "h_chain_seq", "l_chain_seq", "antigen_seqs"])

## Set base path to PDB files
# base_path_to_pdbs = "./pdbs_test"
base_path_to_pdbs = "./test_seqs"


In [4]:
# List all PDB files in test_seqs folder
pdb_files = glob.glob(f"{base_path_to_pdbs}/*.pdb")

for pdb_file in pdb_files:
    pdb_id = os.path.splitext(os.path.basename(pdb_file))[0]
    pdb_df = PandasPdb().read_pdb(pdb_file)
    available_chains = set(str(c).strip() for c in pdb_df.df['ATOM']['chain_id'].unique())
    # Heuristic: assign first two chains as H and L, rest as antigens
    chain_list = list(available_chains)
    if len(chain_list) < 3:
        print(f"{pdb_id}: Not enough chains found. Skipping.")
        continue
    h_chain_id = chain_list[0]
    l_chain_id = chain_list[1]
    antigen_ids = chain_list[2:]
    print(f"Processing {pdb_file} | H: {h_chain_id}, L: {l_chain_id}, Antigen(s): {antigen_ids}")
    h_chain_seq = extract_sequence_from_pdb(pdb_df, h_chain_id, record_name='ATOM')
    l_chain_seq = extract_sequence_from_pdb(pdb_df, l_chain_id, record_name='ATOM')
    antigen_seqs = ''
    for antigen_id in antigen_ids:
        antigen_seq = extract_sequence_from_pdb(pdb_df, antigen_id, record_name='ATOM')
        antigen_seqs += antigen_seq + '|'
    sequence_df_row = pd.DataFrame({
        "pdb_id": pdb_id,
        "h_chain_id": h_chain_id,
        "l_chain_id": l_chain_id,
        "antigen_ids": '|'.join(antigen_ids),
        "h_chain_seq": h_chain_seq,
        "l_chain_seq": l_chain_seq,
        "antigen_seqs": antigen_seqs.strip('|')
    }, index=[0])
    sequences_df = pd.concat([sequences_df, sequence_df_row], ignore_index=True)

1peb: Not enough chains found. Skipping.
2wh6: Not enough chains found. Skipping.
3di3: Not enough chains found. Skipping.
Processing ./test_seqs\4oo8.pdb | H: C, L: B, Antigen(s): ['D', 'F', 'A', 'E']
Processing ./test_seqs\4oo8.pdb | H: C, L: B, Antigen(s): ['D', 'F', 'A', 'E']
4z18: Not enough chains found. Skipping.
Processing ./test_seqs\8hgo.pdb | H: A, L: C, Antigen(s): ['B']
4z18: Not enough chains found. Skipping.
Processing ./test_seqs\8hgo.pdb | H: A, L: C, Antigen(s): ['B']
ma-c7d6n: Not enough chains found. Skipping.
Processing ./test_seqs\ma-c8s4s.pdb | H: H, L: P, Antigen(s): ['L']
ma-c7d6n: Not enough chains found. Skipping.
Processing ./test_seqs\ma-c8s4s.pdb | H: H, L: P, Antigen(s): ['L']
Processing ./test_seqs\ma-cdl3l.pdb | H: H, L: A, Antigen(s): ['L']
Processing ./test_seqs\ma-cdl3l.pdb | H: H, L: A, Antigen(s): ['L']
ma-ci9yk: Not enough chains found. Skipping.
ma-cjnxd: Not enough chains found. Skipping.
ma-cru5u: Not enough chains found. Skipping.
ma-ci9yk: No

In [ ]:
sequences_df.to_csv("sabdab_test_sequences.csv", index=False)

In [ ]:
sequences_df = pd.read_csv("sabdab_test_sequences.csv")
base_path_to_pdbs = "./test_seqs"

In [ ]:
def get_epitope_residues_pandaprot(pdb_file, h_chain_id, l_chain_id, antigen_ids):
    try:
        # Decompress if needed
        if pdb_file.endswith('.gz'):
            import gzip, shutil
            temp_pdb = pdb_file[:-3]
            with gzip.open(pdb_file, 'rt') as f_in, open(temp_pdb, 'w') as f_out:
                shutil.copyfileobj(f_in, f_out)
            pdb_path = temp_pdb
        else:
            pdb_path = pdb_file

        # Get available chains
        pdb_df = PandasPdb().read_pdb(pdb_path)
        available_chains = set(str(c).strip() for c in pdb_df.df['ATOM']['chain_id'].unique())

        # Only use chains that are present
        h_chain_id = h_chain_id if h_chain_id in available_chains else None
        l_chain_id = l_chain_id if l_chain_id in available_chains else None
        antigen_ids = [c for c in antigen_ids if c in available_chains]
        chains = [c for c in [h_chain_id, l_chain_id] if c] + antigen_ids

        if not h_chain_id or not l_chain_id or not antigen_ids:
            print(f"Skipping {pdb_file}: Required chains not found. Available: {available_chains}")
            if pdb_file.endswith('.gz'):
                os.remove(temp_pdb)
            return []

        analyzer = PandaProt(pdb_path, chains=chains)
        interactions = analyzer.map_interactions()

        epitope_residues = []
        relevant_interactions = []
        for interaction_type, interactions_list in interactions.items():
            for interaction in interactions_list:
                chain1 = interaction.get('chain1', interaction.get('donor_chain', ''))
                chain2 = interaction.get('chain2', interaction.get('acceptor_chain', ''))
                res1 = interaction.get('residue1', interaction.get('donor_residue', ''))
                res2 = interaction.get('residue2', interaction.get('acceptor_residue', ''))
                # Only consider antigen-antibody interactions
                for antigen_id in antigen_ids:
                    if (
                        (chain1 == antigen_id and chain2 in [h_chain_id, l_chain_id]) or
                        (chain2 == antigen_id and chain1 in [h_chain_id, l_chain_id])
                    ):
                        epitope_residues.append(f"{antigen_id}:{res1 if chain1 == antigen_id else res2}")
                        relevant_interactions.append((interaction_type, interaction))
                    # Ignore antigen-antigen interactions

        # Print only relevant interactions
        # if relevant_interactions:
        #     print(f"Relevant interactions for {os.path.basename(pdb_file)}:")
        #     for interaction_type, interaction in relevant_interactions:
        #         print(f"  - {interaction_type}: {interaction}")
        # else:
        #     print(f"No relevant interactions found for {os.path.basename(pdb_file)}.")

        if pdb_file.endswith('.gz'):
            os.remove(temp_pdb)
        return sorted(set(epitope_residues))
    except Exception as e:
        print(f"Error processing {pdb_file}: {e}")
        return []

# # Run on the first 30 entries
# for idx, row in sequences_df.head(30).iterrows():
#     pdb_file = f"{base_path_to_pdbs}/{row['pdb_id']}.pdb.gz"
#     h_chain_id = row['h_chain_id']
#     l_chain_id = row['l_chain_id']
#     antigen_ids = [c.strip() for c in row['antigen_ids'].split('|')]
#     residues = get_epitope_residues_pandaprot(pdb_file, h_chain_id, l_chain_id, antigen_ids)
#     print(f"{row['pdb_id']} epitope residues: {', '.join(residues) if residues else 'None found'}")

In [ ]:
def build_resnum_to_seq_idx_map(pdb_file, chain_id):
    """
    Returns a dict mapping PDB residue numbers (as int) to sequence indices (1-based) for a given chain.
    """
    pdb = PandasPdb().read_pdb(pdb_file)
    atom_df = pdb.df['ATOM']
    # Only keep rows for the specified chain
    chain_df = atom_df[atom_df['chain_id'] == chain_id]
    # Get unique residue numbers in order of appearance
    residues = chain_df[['residue_number', 'residue_name']].drop_duplicates()
    resnum_to_idx = {}
    for idx, (resnum, _) in enumerate(residues.values, 1):  # 1-based index
        resnum_to_idx[int(resnum)] = idx
    return resnum_to_idx

In [ ]:
import re

In [ ]:
def highlight_epitope_in_sequence(sequence, chain_id, epitope_residues, resnum_to_idx):
    """
    Places brackets around epitope residues in the antigen sequence.
    sequence: str, full antigen sequence (1-letter code)
    chain_id: str, chain identifier (e.g., 'A')
    epitope_residues: list of str, e.g., ['A:ARG 176', ...]
    resnum_to_idx: dict mapping PDB residue numbers to sequence indices (1-based)
    """
    import re
    pattern = re.compile(rf"^{chain_id}:(?:\w+)\s*(\d+)$")
    epitope_seq_indices = set()
    for res in epitope_residues:
        m = pattern.match(res)
        if m:
            pdb_resnum = int(m.group(1))
            seq_idx = resnum_to_idx.get(pdb_resnum)
            if seq_idx:
                epitope_seq_indices.add(seq_idx)
    highlighted = ""
    for i, aa in enumerate(sequence, 1):
        if i in epitope_seq_indices:
            highlighted += f"[{aa}]"
        else:
            highlighted += aa
    return highlighted

# Collect results for all rows
combined_results = []
for idx, row in sequences_df.iterrows():
    pdb_file = f"{base_path_to_pdbs}/{row['pdb_id']}.pdb.gz"
    h_chain_id = row['h_chain_id']
    l_chain_id = row['l_chain_id']
    antigen_ids = [c.strip() for c in row['antigen_ids'].split('|')]
    residues = get_epitope_residues_pandaprot(pdb_file, h_chain_id, l_chain_id, antigen_ids)
    antigen_seqs = str(row['antigen_seqs']).split('|') if pd.notnull(row['antigen_seqs']) else []
    chain_list, seq_list, res_list = [], [], []
    for i, antigen_chain in enumerate(antigen_ids):
        antigen_sequence = antigen_seqs[i] if i < len(antigen_seqs) else None
        if antigen_sequence and antigen_sequence != 'nan':
            try:
                resnum_to_idx = build_resnum_to_seq_idx_map(pdb_file, antigen_chain)
                highlighted_seq = highlight_epitope_in_sequence(antigen_sequence, antigen_chain, residues, resnum_to_idx)
            except Exception as e:
                print(f"Error mapping for {row['pdb_id']} chain {antigen_chain}: {e}")
                highlighted_seq = None
        else:
            highlighted_seq = None
        chain_list.append(antigen_chain)
        seq_list.append(highlighted_seq if highlighted_seq else "")
        # Only include residues for this chain
        chain_residues = [r for r in residues if r.startswith(f"{antigen_chain}:")]
        res_list.append('|'.join(chain_residues))
    combined_results.append({
        'pdb_id': row['pdb_id'],
        'antigen_chains': '|'.join(chain_list),
        'highlighted_epitope_sequences': '|'.join(seq_list),
        'epitope_residues': '|'.join(res_list)
    })

# Create DataFrame and merge with original
highlight_df = pd.DataFrame(combined_results)
# Merge on pdb_id and antigen_chain (if you want to keep all original columns)
# merged_df = pd.merge(sequences_df, highlight_df, left_on=['pdb_id'], right_on=['pdb_id'], how='left')

# Save to new CSV (recommended to avoid overwriting original)
# merged_df.to_csv("sabdab_sequences_with_epitope.csv", index=False)

In [ ]:
highlight_df.to_csv("sabdab_testSeqs_highlighted_epitopes.csv", index=False)